Здесь мы протестируем модель

In [3]:
!pip install onnxruntime

In [4]:
import onnxruntime as rt

In [5]:
import pandas as pd

In [6]:
df_test = pd.read_csv('cs-test.csv')
df_test.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,NaN,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0
1,2,NaN,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0
2,3,NaN,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0
3,4,NaN,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0
4,5,NaN,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0


In [7]:
df_test = df_test.drop(['SeriousDlqin2yrs'], axis=1)
df_test.dropna().shape

(81400, 11)

In [8]:
df_test.dropna(inplace=True)

Честно, я не успевал доделать и решил просто удалить пустые значения

In [9]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [10]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df_test)
X_scaled = X_scaled.astype(np.float32)

In [11]:
y_df = pd.read_csv('sampleEntry.csv')
y_df.head()

,Id,Probability
0,1,0.080807
1,2,0.040719
2,3,0.011968
3,4,0.067640
4,5,0.108264


In [20]:
y = y_df['Probability']

In [21]:
def to_categorical_test(y) -> list:
    return [y, 1-y]

In [22]:
y = y.map(to_categorical_test)
y[0]

[0.080806647, 0.919193353]

In [12]:
model_path = 'model.onnx'
session = rt.InferenceSession(model_path)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

input_data = X_scaled

output = session.run([output_name], {input_name: input_data})


In [24]:
output

[array([[nan, nan],
        [nan, nan],
        [nan, nan],
        ...,
        [nan, nan],
        [nan, nan],
        [nan, nan]], dtype=float32)]

In [25]:
def from_categorical(vec: list) -> int:
    if vec[0] > vec[1]:
        return 1
    else:
        return 0

In [31]:
y.tolist

<bound method IndexOpsMixin.tolist of 0                [0.080806647, 0.919193353]
1         [0.040718741, 0.9592812589999999]
2                [0.011968341, 0.988031659]
3                [0.067639933, 0.932360067]
4                [0.108263859, 0.891736141]
                        ...                
101498           [0.045363393, 0.954636607]
101499           [0.343775213, 0.656224787]
101500           [0.006969907, 0.993030093]
101501           [0.121993557, 0.878006443]
101502           [0.044248408, 0.955751592]
Name: Probability, Length: 101503, dtype: object>

In [32]:
y_real = y.map(from_categorical)
y_pred = output.map(from_categorical)

AttributeError: 'list' object has no attribute 'map'

Не успел доделать :(